In [1]:
import time
from fox_submission_solver import FoxSolution
from helpers import save_logs

In [2]:
fox = FoxSolution(LOGGING=True)

Loading Stegano model...
Model loaded.


In [3]:
fox.remaining_attempts()

{'error': 'Invalid teamId'} 400


In [4]:
fox.api_base_url = "http://16.171.171.147:5000/fox"

In [5]:
fox.submit_fox_attempt()

Starting Fox Game...
{"carrier_image":[[[31,57,48],[24,47,41],[25,43,43],[43,52,57],[37,32,39],[43,23,32],[77,38,43],[72,21,20],[86,21,15],[96,28,15],[102,36,22],[90,34,19],[82,44,33],[79,56,48],[70,65,62],[32,38,38],[11,26,29],[11,30,34],[12,31,35],[1,20,24]],[[15,34,28],[18,34,31],[29,40,42],[23,26,33],[56,46,55],[84,61,71],[74,33,39],[99,44,47],[102,35,29],[125,55,45],[131,63,50],[142,84,72],[163,120,111],[164,139,134],[151,141,140],[79,83,86],[18,32,35],[12,31,35],[12,31,35],[4,23,27]],[[15,24,23],[23,29,29],[52,53,58],[101,94,102],[114,96,108],[89,59,71],[97,51,61],[131,72,76],[161,91,91],[129,54,49],[177,104,97],[200,137,128],[168,121,115],[173,139,138],[166,147,151],[45,40,46],[33,44,48],[14,33,37],[11,30,34],[6,25,29]],[[14,18,19],[25,26,28],[30,27,34],[77,67,76],[105,87,99],[105,77,89],[128,87,95],[135,83,87],[127,63,63],[131,63,60],[146,81,75],[142,83,77],[166,121,118],[116,84,87],[59,38,45],[60,55,62],[50,61,65],[17,36,40],[8,27,31],[7,26,30]],[[25,31,31],[31,37,37],[30,31,3

In [6]:
save_logs("fox_logs/fox_logs9.txt")